<a href="https://www.kaggle.com/code/vtrackstar/hooper-island-fantasy-basketball?scriptVersionId=270052149" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install espn-api

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from espn_api.basketball import League

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
google-cloud-bigtable 2.32.0 requires google-api-core[grpc]<3.0.0,>=2.17.0, but you have google-api-core 1.34.1 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is in

In [2]:
# League Info
league_id = 2072742117
season_id = 2026
swid = '{A5D0418F-EB37-4A5B-9041-8FEB372A5BCC}'
espn_s2 = 'AECpVxhDRvfrKeWJdI9zJkUUbpoBPS5ybsG8SLFsL%2BIB%2ByM0zKt0xT2qU%2Bhoa%2B05NvvO%2BSkz3SK7VkToD4Yw5fg89Vrj8qPB%2FGHC3W81lJ9bWPTUw3Ss9YSFuYUIkorKnXJBs3YefGPPo4n9SabSdlY0YbEx2oQ%2Bdls8e3JcKMVHXf4j9ER3v1qY406MEWmlIc0Ubw2u7cEjs1yUGWlOPfMcZml%2BTUkTVF6HJhEjzWQ6bhOGUHEif8KfpJ4TY1ydkf8l%2F5JycpD9m1cQ8bP9f4g5b5EqmF4U0ZxHrM6FFc%2FPSw%3D%3D'

league = League(league_id=league_id, year=season_id, espn_s2=espn_s2, swid=swid)

In [3]:
print("League Name:", league.settings.name)
for team in league.teams:
    print(team.team_name, "-", [p.name for p in team.roster])

League Name: My 2024 League
Geriatric Knee's and Buckets - ['Cade Cunningham', 'Josh Giddey', 'Kevin Durant', 'Chet Holmgren', 'Jaren Jackson Jr.', 'Jimmy Butler III', 'Draymond Green', 'Kawhi Leonard', 'RJ Barrett', "Kel'el Ware", 'Cam Thomas', 'Ausar Thompson', 'Christian Braun']
TytheGreat - ['Donovan Mitchell', 'Bam Adebayo', 'Jalen Williams', 'Dyson Daniels', 'Miles Bridges', 'Anfernee Simons', 'Ivica Zubac', 'Zach LaVine', 'Julius Randle', "De'Aaron Fox", 'Jaden McDaniels', 'Paul George', 'Immanuel Quickley']
Da Man Dem - ['Giannis Antetokounmpo', 'Amen Thompson', 'James Harden', 'Evan Mobley', 'Norman Powell', 'Jordan Poole', 'Zion Williamson', 'DeMar DeRozan', 'Payton Pritchard', 'Deandre Ayton', 'Andrew Nembhard', 'Ace Bailey', 'Reed Sheppard']
Real Njggas do Real Things - ['Anthony Edwards', 'Stephen Curry', 'Karl-Anthony Towns', 'Jaylen Brown', 'Desmond Bane', 'Cooper Flagg', 'Nikola Vucevic', 'Trey Murphy III', 'Brandon Miller', 'Kevin Porter Jr.', 'CJ McCollum', 'John Coll

In [4]:
team_list = []
player_list = []

for team in league.teams:
    for player in team.roster:
        team_list.append(team.team_name)
        player_list.append(player.name)

fantasydf = pd.DataFrame({
    "Team": team_list,
    "Player": player_list
})

fantasydf

,Team,Player
0,Geriatric Knee's and Buckets,Cade Cunningham
1,Geriatric Knee's and Buckets,Josh Giddey
2,Geriatric Knee's and Buckets,Kevin Durant
3,Geriatric Knee's and Buckets,Chet Holmgren
4,Geriatric Knee's and Buckets,Jaren Jackson Jr.
...,...,...
101,Dev,Naz Reid
102,Dev,Coby White
103,Dev,D'Angelo Russell
104,Dev,Nic Claxton


In [5]:
url = "https://www.basketball-reference.com/leagues/NBA_2026_totals.html#totals_stats"
tables = pd.read_html(url)
season_stats = tables[0]
print(season_stats.head(), season_stats.columns)

    Rk                   Player   Age Team Pos    G   GS    MP    FG   FGA  \
0  1.0              Luka Dončić  26.0  LAL  PG  1.0  1.0  41.0  17.0  27.0   
1  2.0           Alperen Şengün  23.0  HOU   C  1.0  1.0  49.0  12.0  24.0   
2  3.0  Shai Gilgeous-Alexander  27.0  OKC  PG  1.0  1.0  47.0  12.0  26.0   
3  4.0             Jimmy Butler  36.0  GSW  SF  1.0  1.0  35.0   7.0  14.0   
4  5.0            Chet Holmgren  23.0  OKC   C  1.0  1.0  39.0  11.0  17.0   

   ...   DRB   TRB  AST  STL  BLK  TOV   PF   PTS  Trp-Dbl  Awards  
0  ...  12.0  12.0  9.0  2.0  1.0  3.0  1.0  43.0      0.0     NaN  
1  ...   7.0  11.0  7.0  2.0  1.0  4.0  5.0  39.0      0.0     NaN  
2  ...   5.0   5.0  5.0  2.0  2.0  3.0  2.0  35.0      0.0     NaN  
3  ...   3.0   5.0  4.0  1.0  0.0  1.0  4.0  31.0      0.0     NaN  
4  ...   5.0   7.0  2.0  0.0  0.0  3.0  6.0  28.0      0.0     NaN  

[5 rows x 32 columns] Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P',

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [6]:
szn = season_stats[['Player', 'Age', 'Team', 'Pos', 'MP', 'FG', 'FGA',
       '3P', 'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'PTS']]
szn

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Player,Age,Team,Pos,MP,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
0,Luka Dončić,26.0,LAL,PG,41.0,17.0,27.0,2.0,7.0,10.0,12.0,9.0,2.0,1.0,43.0
1,Alperen Şengün,23.0,HOU,C,49.0,12.0,24.0,5.0,10.0,11.0,11.0,7.0,2.0,1.0,39.0
2,Shai Gilgeous-Alexander,27.0,OKC,PG,47.0,12.0,26.0,1.0,10.0,14.0,5.0,5.0,2.0,2.0,35.0
3,Jimmy Butler,36.0,GSW,SF,35.0,7.0,14.0,1.0,16.0,16.0,5.0,4.0,1.0,0.0,31.0
4,Chet Holmgren,23.0,OKC,C,39.0,11.0,17.0,2.0,4.0,4.0,7.0,2.0,0.0,0.0,28.0
5,Austin Reaves,27.0,LAL,SG,36.0,9.0,16.0,1.0,7.0,10.0,5.0,9.0,2.0,0.0,26.0
6,Stephen Curry,37.0,GSW,PG,32.0,6.0,14.0,3.0,8.0,8.0,1.0,4.0,3.0,1.0,23.0
7,Kevin Durant,37.0,HOU,PF,47.0,9.0,16.0,0.0,5.0,6.0,9.0,3.0,0.0,0.0,23.0
8,Amen Thompson,23.0,HOU,SF,39.0,8.0,17.0,0.0,2.0,2.0,4.0,5.0,1.0,1.0,18.0
9,Buddy Hield,33.0,GSW,SG,22.0,6.0,11.0,5.0,0.0,0.0,1.0,0.0,1.0,0.0,17.0


In [7]:
totals = fantasydf.merge(szn, how='left', on='Player')
totals

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Team_x,Player,Age,Team_y,Pos,MP,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
0,Geriatric Knee's and Buckets,Cade Cunningham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geriatric Knee's and Buckets,Josh Giddey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Geriatric Knee's and Buckets,Kevin Durant,37.0,HOU,PF,47.0,9.0,16.0,0.0,5.0,6.0,9.0,3.0,0.0,0.0,23.0
3,Geriatric Knee's and Buckets,Chet Holmgren,23.0,OKC,C,39.0,11.0,17.0,2.0,4.0,4.0,7.0,2.0,0.0,0.0,28.0
4,Geriatric Knee's and Buckets,Jaren Jackson Jr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Dev,Naz Reid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,Dev,Coby White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,Dev,D'Angelo Russell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Dev,Nic Claxton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
!pip install rapidfuzz
from rapidfuzz import process, fuzz
fantasydf['Player_clean'] = fantasydf['Player'].str.replace(r'\*', '', regex=True).str.lower()
szn['Player_clean'] = szn['Player'].str.replace(r'\*', '', regex=True).str.lower()

# Function to find the closest match from szn for a given fantasy player
def get_best_match(name, choices, score_cutoff=80):
    match = process.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    if match and match[1] >= score_cutoff:
        return match[0], match[1]
    else:
        return None, None

# Apply mapping
matches = []
for player in fantasydf['Player_clean']:
    best_match, score = get_best_match(player, szn['Player_clean'])
    matches.append({'Fantasy_Player': player, 'Best_Szn_Match': best_match, 'Score': score})

matches_df = pd.DataFrame(matches)

# Players with no good match
unmatched = matches_df[matches_df['Best_Szn_Match'].isna()]
print("Unmatched players:\n", unmatched)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.4 MB/s eta 0:00:00
Unmatched players:
         Fantasy_Player Best_Szn_Match  Score
0      cade cunningham           None    NaN
1          josh giddey           None    NaN
4    jaren jackson jr.           None    NaN
7        kawhi leonard           None    NaN
8           rj barrett           None    NaN
..                 ...            ...    ...
101           naz reid           None    NaN
102         coby white           None    NaN
103   d'angelo russell           None    NaN
104        nic claxton           None    NaN
105      collin sexton           None    NaN

[92 rows x 3 columns]


/tmp/ipykernel_13/3212877986.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  szn['Player_clean'] = szn['Player'].str.replace(r'\*', '', regex=True).str.lower()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [9]:
matches

[{'Fantasy_Player': 'cade cunningham', 'Best_Szn_Match': None, 'Score': None},
 {'Fantasy_Player': 'josh giddey', 'Best_Szn_Match': None, 'Score': None},
 {'Fantasy_Player': 'kevin durant',
  'Best_Szn_Match': 'kevin durant',
  'Score': 100.0},
 {'Fantasy_Player': 'chet holmgren',
  'Best_Szn_Match': 'chet holmgren',
  'Score': 100.0},
 {'Fantasy_Player': 'jaren jackson jr.',
  'Best_Szn_Match': None,
  'Score': None},
 {'Fantasy_Player': 'jimmy butler iii',
  'Best_Szn_Match': 'jimmy butler',
  'Score': 85.71428571428572},
 {'Fantasy_Player': 'draymond green',
  'Best_Szn_Match': 'draymond green',
  'Score': 100.0},
 {'Fantasy_Player': 'kawhi leonard', 'Best_Szn_Match': None, 'Score': None},
 {'Fantasy_Player': 'rj barrett', 'Best_Szn_Match': None, 'Score': None},
 {'Fantasy_Player': "kel'el ware", 'Best_Szn_Match': None, 'Score': None},
 {'Fantasy_Player': 'cam thomas', 'Best_Szn_Match': None, 'Score': None},
 {'Fantasy_Player': 'ausar thompson', 'Best_Szn_Match': None, 'Score': None

In [10]:
fantasydf1 = fantasydf.merge(matches_df, left_on='Player_clean', right_on='Fantasy_Player', how='left')
print(fantasydf1.head())

                           Team             Player       Player_clean  \
0  Geriatric Knee's and Buckets    Cade Cunningham    cade cunningham   
1  Geriatric Knee's and Buckets        Josh Giddey        josh giddey   
2  Geriatric Knee's and Buckets       Kevin Durant       kevin durant   
3  Geriatric Knee's and Buckets      Chet Holmgren      chet holmgren   
4  Geriatric Knee's and Buckets  Jaren Jackson Jr.  jaren jackson jr.   

      Fantasy_Player Best_Szn_Match  Score  
0    cade cunningham           None    NaN  
1        josh giddey           None    NaN  
2       kevin durant   kevin durant  100.0  
3      chet holmgren  chet holmgren  100.0  
4  jaren jackson jr.           None    NaN  


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [11]:
fantasyszn = fantasydf1.merge(szn, left_on='Best_Szn_Match', right_on='Player_clean', how='left')
print(fantasyszn.head(), fantasyszn.columns)

                         Team_x           Player_x     Player_clean_x  \
0  Geriatric Knee's and Buckets    Cade Cunningham    cade cunningham   
1  Geriatric Knee's and Buckets        Josh Giddey        josh giddey   
2  Geriatric Knee's and Buckets       Kevin Durant       kevin durant   
3  Geriatric Knee's and Buckets      Chet Holmgren      chet holmgren   
4  Geriatric Knee's and Buckets  Jaren Jackson Jr.  jaren jackson jr.   

      Fantasy_Player Best_Szn_Match  Score       Player_y   Age Team_y  Pos  \
0    cade cunningham           None    NaN            NaN   NaN    NaN  NaN   
1        josh giddey           None    NaN            NaN   NaN    NaN  NaN   
2       kevin durant   kevin durant  100.0   Kevin Durant  37.0    HOU   PF   
3      chet holmgren  chet holmgren  100.0  Chet Holmgren  23.0    OKC    C   
4  jaren jackson jr.           None    NaN            NaN   NaN    NaN  NaN   

   ...   FGA   3P   FT  FTA  TRB  AST  STL  BLK   PTS  Player_clean_y  
0  ...   NaN  

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [12]:
fantasystats = fantasyszn[['Team_x', 'Fantasy_Player', 'Age', 'Team_y', 'Pos', 'FG',
       'FGA', '3P', 'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'PTS']]
fantasystats

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Team_x,Fantasy_Player,Age,Team_y,Pos,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
0,Geriatric Knee's and Buckets,cade cunningham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geriatric Knee's and Buckets,josh giddey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Geriatric Knee's and Buckets,kevin durant,37.0,HOU,PF,9.0,16.0,0.0,5.0,6.0,9.0,3.0,0.0,0.0,23.0
3,Geriatric Knee's and Buckets,chet holmgren,23.0,OKC,C,11.0,17.0,2.0,4.0,4.0,7.0,2.0,0.0,0.0,28.0
4,Geriatric Knee's and Buckets,jaren jackson jr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Dev,naz reid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,Dev,coby white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,Dev,d'angelo russell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Dev,nic claxton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
fantasystats.columns

Index(['Team_x', 'Fantasy_Player', 'Age', 'Team_y', 'Pos', 'FG', 'FGA', '3P',
       'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'PTS'],
      dtype='object')

In [14]:
team_agg = fantasystats.groupby('Team_x').mean(numeric_only=True).round(2)
team_agg

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Age,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
Team_x,,,,,,,,,,,
Belt,25.00,12.00,25.00,3.00,10.0,12.50,8.0,6.0,2.00,1.5,37.00
Da Man Dem,23.67,5.33,11.67,0.67,1.0,2.00,4.0,3.0,0.67,1.0,12.33
Dade County Snipers,26.50,13.00,21.50,1.50,7.0,10.00,8.5,9.0,2.00,0.5,34.50
Dev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Geriatric Knee's and Buckets,32.75,7.50,13.25,1.00,6.5,7.25,7.0,4.5,0.50,0.5,22.50
Real Njggas do Real Things,32.00,4.00,9.50,1.50,5.0,5.00,4.5,4.5,2.50,0.5,14.50
Teague Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TytheGreat,23.00,1.00,5.00,0.00,0.0,0.00,0.0,1.0,1.00,0.0,2.00


In [15]:
week1 = fantasystats.copy()

week1 = week1.drop(columns=['Pos', 'Age'])
week1 = week1.rename(columns={'Fantasy_Player': 'Player', 'Team_x': 'Fantasy_Team', 'Team_y': 'NBA_Team'})

week1 = week1[['Fantasy_Team', 'Player', 'NBA_Team', 'FG', 'FGA', '3P', 'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'PTS']]

print(week1.head(), week1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Fantasy_Team  106 non-null    object 
 1   Player        106 non-null    object 
 2   NBA_Team      14 non-null     object 
 3   FG            14 non-null     float64
 4   FGA           14 non-null     float64
 5   3P            14 non-null     float64
 6   FT            14 non-null     float64
 7   FTA           14 non-null     float64
 8   TRB           14 non-null     float64
 9   AST           14 non-null     float64
 10  STL           14 non-null     float64
 11  BLK           14 non-null     float64
 12  PTS           14 non-null     float64
dtypes: float64(10), object(3)
memory usage: 10.9+ KB
                   Fantasy_Team             Player NBA_Team    FG   FGA   3P  \
0  Geriatric Knee's and Buckets    cade cunningham      NaN   NaN   NaN  NaN   
1  Geriatric Knee's and Buckets     

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [16]:
week1

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Fantasy_Team,Player,NBA_Team,FG,FGA,3P,FT,FTA,TRB,AST,STL,BLK,PTS
0,Geriatric Knee's and Buckets,cade cunningham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geriatric Knee's and Buckets,josh giddey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Geriatric Knee's and Buckets,kevin durant,HOU,9.0,16.0,0.0,5.0,6.0,9.0,3.0,0.0,0.0,23.0
3,Geriatric Knee's and Buckets,chet holmgren,OKC,11.0,17.0,2.0,4.0,4.0,7.0,2.0,0.0,0.0,28.0
4,Geriatric Knee's and Buckets,jaren jackson jr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Dev,naz reid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,Dev,coby white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,Dev,d'angelo russell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Dev,nic claxton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
topweek1 = week1.groupby('Fantasy_Team').sum(numeric_only=True).round(2)
topweek1['FG%'] = topweek1['FG'] / topweek1['FGA']
topweek1['FT%'] = topweek1['FT'] / topweek1['FTA']
print(topweek1.head())

categories = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', 'FT%', '3P']

                                FG   FGA   3P    FT   FTA   TRB   AST  STL  \
Fantasy_Team                                                                 
Belt                          24.0  50.0  6.0  20.0  25.0  16.0  12.0  4.0   
Da Man Dem                    16.0  35.0  2.0   3.0   6.0  12.0   9.0  2.0   
Dade County Snipers           26.0  43.0  3.0  14.0  20.0  17.0  18.0  4.0   
Dev                            0.0   0.0  0.0   0.0   0.0   0.0   0.0  0.0   
Geriatric Knee's and Buckets  30.0  53.0  4.0  26.0  29.0  28.0  18.0  2.0   

                              BLK   PTS       FG%       FT%  
Fantasy_Team                                                 
Belt                          3.0  74.0  0.480000  0.800000  
Da Man Dem                    3.0  37.0  0.457143  0.500000  
Dade County Snipers           1.0  69.0  0.604651  0.700000  
Dev                           0.0   0.0       NaN       NaN  
Geriatric Knee's and Buckets  2.0  90.0  0.566038  0.896552  


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [18]:
rank_df = topweek1.copy()

# Categories to rank
categories = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', 'FT%', '3P']

# Rank each category (1 = best)
for cat in categories:
    rank_df[f'{cat}_Rank'] = rank_df[cat].rank(ascending=False, method='min')

# Sum ranks (lower = better)
rank_df['Sum'] = rank_df[[f'{cat}_Rank' for cat in categories]].sum(axis=1)

# Sort by Sum (best teams on top)
rank_df = rank_df.sort_values('Sum')
rankedwk1 = rank_df[['Sum', 'PTS', 'TRB', 'AST', 'STL', 'BLK', '3P', 'FG%', 'FT%']]
rankedwk1

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Sum,PTS,TRB,AST,STL,BLK,3P,FG%,FT%
Fantasy_Team,,,,,,,,,
Geriatric Knee's and Buckets,16.0,90.0,28.0,18.0,2.0,2.0,4.0,0.566038,0.896552
Belt,18.0,74.0,16.0,12.0,4.0,3.0,6.0,0.480000,0.800000
Dade County Snipers,20.0,69.0,17.0,18.0,4.0,1.0,3.0,0.604651,0.700000
Real Njggas do Real Things,28.0,29.0,9.0,9.0,5.0,1.0,3.0,0.421053,1.000000
Da Man Dem,31.0,37.0,12.0,9.0,2.0,3.0,2.0,0.457143,0.500000
Dev,39.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Teague Time,39.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
TytheGreat,42.0,2.0,0.0,1.0,1.0,0.0,0.0,0.200000,NaN
